In [21]:
import scipy.io as sio
import scipy.stats as stats
import numpy as np
import pandas as pd
from collections import Counter
from sentence_transformers import SentenceTransformer

### 1.Embeddings

In [3]:
model = SentenceTransformer("all-mpnet-base-v2")

In [22]:
# Load data
data = sio.loadmat('chemical_reprogramming_Kimmel/datos_MSX1.mat') 
X = data['X']
# TODO: si hace falta ponerlo como dataframe
Y_df = data['y']
gene_names = pd.read_csv('chemical_reprogramming_Kimmel/names_msx1_filtrado.csv')

print(f"X dim: {X.shape}")
print(f"Y dim: {Y.shape}. Labels: {np.unique(Y)}")
print(f"Number of samples by label: {Counter(Y.reshape(1186,))}")

# Remove columns where mode is repeated more than 95% of the data
mode_values, mode_counts = stats.mode(X, axis=0)
cols_to_keep = np.sum(mode_values == X, axis=0) / X.shape[0] < 0.95
X = X[:, cols_to_keep]

print(f"New X dim: {X.shape}")

X dim: (1186, 22767)
Y dim: (1186, 1). Labels: [-1  1]
Number of samples by label: Counter({-1: 773, 1: 413})
New X dim: (1186, 11897)


In [27]:
# Construyo un dataframe: cells x genes 
X_df = pd.DataFrame(X, columns=list(gene_names.columns))

In [28]:
X_df

,Aox1,Cnnm3,Lman2l,Ankrd39,Usf1,Rrp15,Ddx18,Rgs20,Disp1,Cnot11,...,Gm26775,Jpx,Gm26593,Gm28268,Kdm5d,Uty,Ddx3y,Eif2s3y,Uba1y,Erdr1
0,0,1,0,3,4,5,5,4,0,3,...,0,0,0,0,0,0,3,3,0,5
1,0,1,0,1,0,6,5,0,0,2,...,0,0,0,0,0,3,3,6,0,3
2,1,0,0,1,1,2,9,4,1,3,...,0,1,0,0,0,3,3,1,0,3
3,0,0,0,0,1,0,2,0,0,0,...,0,0,0,0,1,2,4,3,0,0
4,0,2,1,0,5,4,3,0,1,2,...,0,0,0,0,0,2,5,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,0,0,0,0,0,0,5,0,0,1,...,0,0,0,1,0,0,1,0,0,0
1182,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1183,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1184,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [31]:
# Encoder one sample
# ASÍ NO PUEDO CALCULAR EMBEDDINGS PORQUE SON NÚMEROS, QUITO LOS 0 Y USO LOS NOMBRES? 
# SERVIRÍA DE ALGO TENER LAS POSICIONES, ES DECIR, LOS GENES ORDENADOS?

# Sería como una matriz de co-ocurrencia, es decir, que representa cuantas veces aparece la palabra en el documento, que 
# interpolado sería cuántas veces aparece el gen en la célula

# https://satijalab.org/seurat/ SEURAT, PAQUETE PARA EL ANÁLSISI DE DATOS MULTIÓMICOS EN R
sample = X[:1]
embeddings = model.encode(list(sample))

TypeError: object of type 'numpy.uint8' has no len()

### 2.PCA

In [32]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.41336359 0.0887613 ]
[15180.26970497  7034.37006569]
